In [4]:
import pandas as pd
import numpy as np
import random
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.environ import SolverFactory
import yfinance as yf
import matplotlib.pyplot as plt


In [20]:
model = pyo.ConcreteModel()

#Definir SET
model.MODELOS = pyo.Set(initialize = ['Standart','Luxo'])
model.EQUIPAMENTOS = pyo.Set(initialize = ['lixadoras','polidoras'])
# model.RACAO = pyo.Set(initialize = ['A','B'])

#paramentro dicionarios
duracao = {
    'Standart':{
        'lixadoras':1,
        'polidoras':1,
    },
    'Luxo':{
        'lixadoras':1,
        'polidoras':4,
    }
}

quantidade_equipamento = {
    'lixadoras':2,
    'polidoras':3,
}

lucro = {
    'Standart':24,
    'Luxo':34
}

tempo_maximo_semanal = {
    'lixadoras':40,
    'polidoras':40
}


# VARIAVEIS----------------------
model.x = pyo.Var(model.MODELOS, bounds=(0,None), domain = pyo.NonNegativeIntegers)


#objetivo
def objetivo1(model):

    return sum(
        model.x[m] * lucro[m] for m in model.MODELOS
    )

model.objetivo = pyo.Objective(rule=objetivo1, sense=pyo.maximize)

#Restrição
def necessidade_hora(model,e):

    
    return sum(
        model.x[m] *duracao[m][e] for m in model.MODELOS
        ) <= tempo_maximo_semanal[e] * quantidade_equipamento[e]

model.necessidade_hora = pyo.Constraint(model.EQUIPAMENTOS, rule=necessidade_hora)

In [21]:
model.pprint()

2 Set Declarations
    EQUIPAMENTOS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'lixadoras', 'polidoras'}
    MODELOS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'Standart', 'Luxo'}

1 Var Declarations
    x : Size=2, Index=MODELOS
        Key      : Lower : Value : Upper : Fixed : Stale : Domain
            Luxo :     0 :  None :  None : False :  True : NonNegativeIntegers
        Standart :     0 :  None :  None : False :  True : NonNegativeIntegers

1 Objective Declarations
    objetivo : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 24*x[Standart] + 34*x[Luxo]

1 Constraint Declarations
    necessidade_hora : Size=2, Index=EQUIPAMENTOS, Active=True
        Key       : Lower : Body                    : Upper : Active
        lixadoras :  -Inf :   x[Stand

In [22]:
opt = SolverFactory('cplex', executable='C:\\CPLEX_Studio2211\\cplex\\bin\\x64_win64\\cplex.exe')
opt.options['TimeLimit'] = 180 
results = opt.solve(model, tee=True, warmstart=True)
print("-------FIM----------")



Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\joaon\AppData\Local\Temp\tmpkis6q6fi.cplex.log' open.
CPLEX> New value for time limit in seconds: 180
CPLEX> Problem 'C:\Users\joaon\AppData\Local\Temp\tmpoty3j3zu.pyomo.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> Warning: no MIP start values read, no MIP start loaded.
MIP start file 'C:\Users\joaon\AppData\Local\Temp\tmpwqng8_tx.cplex.mst' read.
CPLEX> Problem name         : C:\Users\joaon\AppData\Local\Temp\tmpoty3j3zu.pyomo.lp
Objective sense      : Maximize
Variables            :       2  [General Integer: 2]
Objective nonzeros   :       2
Linear constraints   :       2  [

In [26]:
pyo.value(model.objetivo)

2050.0